In [ ]:
#подключение к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Обработка текста

In [ ]:
max_n_sent = 10
min_n_sent = 8

In [ ]:
import pandas
df = pandas.read_csv('/content/drive/My Drive/Programm/LeakGAN/datasets/stihi_ru.csv', index_col='author')
print(df[:5])

                                                                   text
author                                                                 
Марина Копсергенова       \r\nДетки мои  Живите неспеша   чтоб пела ...
Владислав Иванов 13   Каждый по своему проводит свою ночь\r\nКто спи...
Лия Эль               Нежно так  я словами касался предплечий\r\nИх ...
Елена Кедровая Шишка  Ты  как всегда  меня поймешь\r\nИ  как всегда ...
Леонид Хегай          Укусить бы кого нибудь больно\r\nиль повыть  п...


In [ ]:
import numpy as np
text_1 = '\n'.join(np.array(df)[:, 0])
print('Всего символов:', len(text_1))

Всего символов: 97431217


In [ ]:
# f = open("/content/drive/My Drive/SeqGAN/datasets/before_preprocessing/corpus_en_ru.txt", "r")
# f = open("/content/drive/My Drive/SeqGAN/datasets/before_preprocessing/corpus.en_ru.1m.en", "r")
f = open("/content/drive/My Drive/SeqGAN/datasets/before_preprocessing/proza_ru_2017_01_short_ru.txt", "r")
text_2 = f.read()#.decode('utf-8')
print("Общая длина текста: ", len(text_2))
text_2[:250]

Общая длина текста:  207139469


'Глава 16\nОКЛЕНД\nВ субботу 10 декабря отправились в Окленд, город напротив Сан-Франциско, через одноименный залив.\nПовёз Дмитрий на своей "Тойоте".\nВ аэропорт Окленда прилетел внук Дмитрия, Костя.\nОн уже прилетал на День Благодарения из Сан-Диего, где'

In [ ]:
# text = text_1 + text_2
text = text_1

In [ ]:
print("Общая длина текста: ", len(text))

Общая длина текста:  97431217


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# в нижний регистр
text = text.lower()

In [ ]:
from copy import deepcopy as dcopy

text_copy = dcopy(text)

In [ ]:
text = dcopy(text_copy)

In [ ]:
# количество упоминаний каждого слова (http://nlpx.net/archives/29)
from collections import Counter

cntr = Counter(nltk.word_tokenize(text)) # подсчет упоминаний

In [ ]:
most_words = list(cntr.most_common(5000-3)) # наиболее частые N-1 слов (добавится заполнитель)
# most_words = cntr.elements() # вернуть слова

In [ ]:
print("Всего слов: ", len(list(cntr.most_common(10000000))))

Всего слов:  456312


In [ ]:
print('Количество упоминаний самого редкого слова до обработки текста:', most_words[-1][0], most_words[-1][1])

Количество упоминаний самого редкого слова до обработки текста: перестала 303


In [ ]:
# создание словаря слов
vocab = dict()
for i in range(len(most_words)):
  vocab[most_words[i][0]] = i
vocab['']=len(vocab) # добавить пробел (заполнитель)
vocab['_UNK_']=len(vocab) # добавить удаленное слово (удаляемые слова заменяются этим словом)
vocab['_START_']=len(vocab) # добавить стартовое слово в словарь

d = vocab
list_d = list(d.items())
list_d.sort(key=lambda i: i[1])

word = []
for i in range(len(list_d)):
  word.append(list_d[i][0])
print('Слов в словаре:', len(word))

Слов в словаре: 5000


In [ ]:
# разделение текста на предложения и слова (если в выборке есть знаки препинания, то примером считается предложение)
# text_processed = []
# for sent in nltk.sent_tokenize(text):
#   text_processed.append(nltk.word_tokenize(sent))

# разделение текста на строки и слова (если в выборке нет знаков препинания или примеры разделены построчно, то примером считается строка)
text_processed = []
sets = text.split('\n') # -|-
for i, sent in enumerate(sets):
  if(i%(len(sets)//10)==0):
    print(float(i)/float(len(sets)), '%')
  text_processed.append(nltk.word_tokenize(sent))

0.0 %
0.09999987573744648 %
0.19999975147489296 %
0.2999996272123394 %
0.3999995029497859 %
0.4999993786872324 %
0.5999992544246788 %
0.6999991301621253 %
0.7999990058995718 %
0.8999988816370184 %
0.9999987573744648 %


In [ ]:
min_len = len(text_processed[0])
max_len = len(text_processed[0])

sr_len = 0
for i in range(len(text_processed)):
  sr_len += len(text_processed[i])
  min_len = min(min_len, len(text_processed[i]))
  max_len = max(max_len, len(text_processed[i]))

print("Предложений: ", len(text_processed))
print("Минимальная длина", min_len)
print("Средняя длина", sr_len/len(text_processed))
print("Максимальная длина", max_len)
print("Количество слов", sr_len)

Предложений:  2414243
Минимальная длина 0
Средняя длина 6.697887495169293
Максимальная длина 1494
Количество слов 16170328


In [ ]:
from copy import deepcopy as dcopy

text_processed_copy = dcopy(text_processed)

In [ ]:
text_processed = dcopy(text_processed_copy)

In [ ]:
# удаление коротких и длиных примеров
DEL_sentences=0
len_texts = len(text_processed)

i=0
ii=0
while i < len(text_processed):
  if(ii%(len_texts//100) == 0):
    print(ii//(len_texts//100), u'В обработке: ', len(text_processed), u"Удалено:", DEL_sentences, u'Сохранено: ', ii-DEL_sentences)
  
  if(len(text_processed[i]) < min_n_sent):# or len(text_processed[i]) > max_n_sent):#
    del(text_processed[i])
    DEL_sentences+=1
  else:
    text_processed[i] = text_processed[i][:max_n_sent]
    i+=1
  ii+=1
print('Предложений: ', len(text_processed)) #112138 #214986 #571377

0 В обработке:  2414243 Удалено: 0 Сохранено:  0
1 В обработке:  2393881 Удалено: 20362 Сохранено:  3780
2 В обработке:  2373153 Удалено: 41090 Сохранено:  7194
3 В обработке:  2352693 Удалено: 61550 Сохранено:  10876
4 В обработке:  2332617 Удалено: 81626 Сохранено:  14942
5 В обработке:  2312144 Удалено: 102099 Сохранено:  18611
6 В обработке:  2291761 Удалено: 122482 Сохранено:  22370
7 В обработке:  2271368 Удалено: 142875 Сохранено:  26119
8 В обработке:  2251085 Удалено: 163158 Сохранено:  29978
9 В обработке:  2230695 Удалено: 183548 Сохранено:  33730
10 В обработке:  2210139 Удалено: 204104 Сохранено:  37316
11 В обработке:  2189664 Удалено: 224579 Сохранено:  40983
12 В обработке:  2169429 Удалено: 244814 Сохранено:  44890
13 В обработке:  2149140 Удалено: 265103 Сохранено:  48743
14 В обработке:  2128665 Удалено: 285578 Сохранено:  52410
15 В обработке:  2108263 Удалено: 305980 Сохранено:  56150
16 В обработке:  2088041 Удалено: 326202 Сохранено:  60070
17 В обработке:  20677

In [ ]:
from copy import deepcopy as dcopy

text_preprocessed_copy = dcopy(text_processed)

In [ ]:
text_processed = dcopy(text_preprocessed_copy)

In [ ]:
# замена не вошедших в словарь слов спец. словом и удаление предложений содержащих много замененных слов
DEL_sentences=0
len_texts = len(text_processed)
i=0
ii=0
while i < len(text_processed):
  if(ii%(len_texts//100) == 0):
    print(ii//(len_texts//100), u'В обработке: ', len(text_processed), u"Удалено:", DEL_sentences, u'Сохранено: ', ii-DEL_sentences)
  
  text_processed[i] = text_processed[i][:max_n_sent] # удаление лишней части предложения

  ln_n = len(text_processed[i])
  ln_s = 0
  j=0
  while j < ln_n:
    if(text_processed[i][j] not in word):
      #del(text_processed[i][j]) continue
      text_processed[i][j] = '_UNK_' #замена слов
      ln_s+=1
    j+=1
  if(ln_s>0):#ln_n*0.05):#если больше, чем N слов в примере не вошло в словарь, удалить пример
    del(text_processed[i])
    DEL_sentences+=1
  else:
    i+=1
  ii+=1
print("Удалено предложений (с недостающими словами):", DEL_sentences)
print('Осталось предложений: ', len(text_processed))

0 В обработке:  378029 Удалено: 0 Сохранено:  0
1 В обработке:  375019 Удалено: 3010 Сохранено:  770
2 В обработке:  372080 Удалено: 5949 Сохранено:  1611
3 В обработке:  369109 Удалено: 8920 Сохранено:  2420
4 В обработке:  366069 Удалено: 11960 Сохранено:  3160
5 В обработке:  363127 Удалено: 14902 Сохранено:  3998
6 В обработке:  360085 Удалено: 17944 Сохранено:  4736
7 В обработке:  357108 Удалено: 20921 Сохранено:  5539
8 В обработке:  354157 Удалено: 23872 Сохранено:  6368
9 В обработке:  351247 Удалено: 26782 Сохранено:  7238
10 В обработке:  348286 Удалено: 29743 Сохранено:  8057
11 В обработке:  345270 Удалено: 32759 Сохранено:  8821
12 В обработке:  342249 Удалено: 35780 Сохранено:  9580
13 В обработке:  339305 Удалено: 38724 Сохранено:  10416
14 В обработке:  336381 Удалено: 41648 Сохранено:  11272
15 В обработке:  333464 Удалено: 44565 Сохранено:  12135
16 В обработке:  330463 Удалено: 47566 Сохранено:  12914
17 В обработке:  327385 Удалено: 50644 Сохранено:  13616
18 В обр

In [ ]:
from copy import deepcopy as dcopy

text_del_processed_copy = dcopy(text_processed)

In [ ]:
text_processed = dcopy(text_del_processed_copy)

In [ ]:
# преобразование текста в набор чисел
for i in range(len(text_processed)):
  for j in range(len(text_processed[i])):
    text_processed[i][j] = vocab[text_processed[i][j]]

In [ ]:
# дополнение предложений до заданной длины заполнителем (' ' - пробелом)
max_n_sent = max_n_sent
for i in range(len(text_processed)):
  while len(text_processed[i]) < max_n_sent:
    text_processed[i].append(vocab[''])

In [ ]:
import pickle
obj = (word, vocab)
output = open('/content/drive/My Drive/SeqGAN/datasets/stihi_ru/stihi_ru_vocab_cotra.pkl', 'wb')
pickle.dump(obj, output, 2)
output.close()

In [ ]:
from copy import deepcopy as dcopy

text_shuffle_copy = dcopy(text_processed)

In [ ]:
text_processed = dcopy(text_shuffle_copy)

In [ ]:
print('Предложений: ', len(text_processed)) #112138 #214986
for i in range(len(text_processed)):
  text_processed[i] = ' '.join(map(str, text_processed[i]))
# print('Предложений: ', len(text_processed)) #112138 #214986
# text_processed = '\n'.join(map(str, text_processed)) + '\n'
# print('Символов: ', len(text_processed)) #15224801 #29791310

Предложений:  78661


In [ ]:
# i=0
# for t in text_processed:
#   if(t=='\n'):
#     i+=1
# print(i, len(text_processed))

In [ ]:
print('\n'.join(map(str, text_processed[:15])) + '\n')

0 168 700 169 489 881 2 1252 4997 4997
46 15 86 1248 9 43 518 11 2287 4997
50 1248 215 149 2 65 4792 0 646 4997
2 1876 716 25 4925 0 2 2422 90 394
66 173 1504 8 905 4 13 31 36 14
31 36 8 606 623 3110 0 1 188 31
2 3311 163 32 683 115 0 2503 4997 4997
0 352 166 53 105 16 28 606 4997 4997
29 1701 174 2 57 10 17 4692 141 2124
189 28 22 1 98 79 136 34 4997 4997
3285 5 4646 10 0 1213 15 955 4997 4997
10 324 5 57 106 113 1392 2 4528 4997
506 1 188 2 694 258 5 3111 233 4997
43 39 489 3899 43 76 1030 150 4997 4997
0 3 206 87 1026 103 17 4692 2612 4997



In [ ]:
from random import shuffle
shuffle(text_processed)

print('Примеров: ', len(text_processed))
text_file = open(u"/content/drive/My Drive/SeqGAN/datasets/stihi_ru/stihi_ru_realtrain_cotra.txt", "w")
text_file.write('\n'.join(map(str, text_processed[:70000])) + '\n')
text_file.close()

text_file = open(u"/content/drive/My Drive/SeqGAN/datasets/stihi_ru/stihi_ru_realtest_coco.txt", "w")
text_file.write('\n'.join(map(str, text_processed[-5000:])) + '\n')
text_file.close()

Примеров:  78661


In [ ]:
print(text_processed[:10])

['0 63 1892 18 68 0 194 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '0 1 98 234 8 11 200 1030 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '86 291 21 97 28 9 60 50 242 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '26 147 5 15 13 1411 3831 2 1867 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '0 2 65 1230 25 44 213 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '174 2 2422 0 194 1 47 34 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '40 204 493 499 4369 1 265 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '546 386 604 18 211 19 570 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '0 14 392 3 7 14 160 880 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997', '1338 8 235 9 20 2 187 7 13 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997 4997']


In [ ]:
# data_Name = "cotra"
# vocab_file = "NEW_vocab_" + data_Name + ".pkl"
# word, vocab = cPickle.load(open('/content/drive/My Drive/Programm/LeakGAN/save/'+vocab_file))

print(vocab[''], vocab['я'], word[-1], word[4990])

4997 3 _START_ лести


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer

snowball = SnowballStemmer(language="russian")

# text = "В частности, он опускает окончания слова.".lower()
text_ = word_tokenize(text[:500000], language="russian")

In [ ]:
def transform_text(text_):
  new_text = ''
  for i in range(len(text_)):
    text_[i] = text_[i].lower()
    text_[i] = text_[i].replace('ё', 'е')
    text_[i] = text_[i].replace('c', 'с').replace('a', 'а').replace('e', 'е').replace('o', 'о')

    stem_word = snowball.stem(text_[i])
    if stem_word != '':
      remainder = text_[i].split(stem_word)
      if remainder[0] != '' and remainder[0] in text_[i]:
        stem_word = remainder[0] + '_ ' + stem_word
      if len(remainder) == 2 and remainder[1] != '' and remainder[1] in text_[i]:
        stem_word = stem_word + ' _' + remainder[1]
      new_text = new_text + ' ' + stem_word
  return new_text

In [ ]:
new_text = ''
for i in range(len(text_processed)):
  if i%(len(text_processed)//10)==0:
    print('.', end='')
  new_text = new_text + '\n' + transform_text(text_processed[i])

.

TypeError: ignored

In [ ]:
from copy import deepcopy as dcopy

text_transform_copy = dcopy(new_text)

In [ ]:
text_processed = dcopy(text_transform_copy)

In [ ]:
text = text_processed

In [ ]:
text_[i], remainder, stem_word

In [ ]:
new_text[:500]